In [1]:
import sys
sys.path.append("../")

In [2]:
import pandas as pd
import datetime as dt
from technicals.indicators import RSI, MACD, ATR, BollingerBands, momentum_reversal
from technicals.patterns import apply_patterns
from simulation.guru_tester import GuruTester
from models.trade_settings import TradeSettings
from constants import defs
import json
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns


In [3]:
BUY = 1
SELL = -1
NONE = 0
RSI_LIMIT = 50.0

def apply_signal(row):

    
    if (
            
            row.ENGULFING == True and
            row.direction == BUY and 
            # row.mid_l > row.EMA_200 and
            # row.RSI_14 > RSI_LIMIT and 
            row.SPREAD <= row.maxspread and
            row.GAIN >= row.mingain and
            # row.mid_c < row.BB_LW and row.mid_o > row.BB_LW and  # Bollinger Bands condition for BUY
            # row[f"RSI_{row.rsi_period}"] < row.rsi_oversold  and # RSI condition for oversold
            row.MACD > row.SIGNAL_MD and
            row.Momentum_Signal == 'Buy'  
            # row.BB_Signal == 'Buy'

         
        ):
       
        return BUY
    if (
            row.direction == SELL and 
            # row.mid_h < row.EMA_200 and
            # row.RSI_14 < RSI_LIMIT and 
            row.SPREAD <= row.maxspread and
            row.GAIN >= row.mingain and
            # row.mid_c > row.BB_UP and row.mid_o < row.BB_UP and  # Bollinger Bands condition for SELL
            # row[f"RSI_{row.rsi_period}"] > row.rsi_overbought and # RSI condition for overbought
            # row.MACD < row.SIGNAL_MD  and
            row.Momentum_Signal == 'Sell'  
            # row.BB_Signal == 'Sell'
        ):
        # print(row)
        return SELL
    
    return NONE        

In [4]:
def apply_SL(row):
    """
    Calculates the Stop Loss (SL) using ATR for dynamic adjustment based on volatility.
    """
    if row.SIGNAL == defs.BUY:
        return row.mid_c - (row[f"ATR_{row.atr_period}"] * row.atr_multiplier)
    elif row.SIGNAL == defs.SELL:
        return row.mid_c + (row[f"ATR_{row.atr_period}"] * row.atr_multiplier)
    return 0.0


def apply_TP(row):
    """
    Calculates the Take Profit (TP) using ATR and risk-reward ratio.
    """
    if row.SIGNAL == defs.BUY:
        return row.mid_c + (row[f"ATR_{row.atr_period}"] * row.atr_multiplier * row.riskreward)
    elif row.SIGNAL == defs.SELL:
        return row.mid_c - (row[f"ATR_{row.atr_period}"] * row.atr_multiplier * row.riskreward)
    return 0.0

In [ ]:
df_an = pd.read_pickle(f"../data/EUR_USD_H1.pkl")
print(df_an.head())


In [6]:

def run_pair(pair):
        df_an = pd.read_pickle(f"../data/{pair}_H1.pkl")
        df_m5 = pd.read_pickle(f"../data/{pair}_M15.pkl")
        df_an.reset_index(drop=True, inplace=True)
        df_m5.reset_index(drop=True, inplace=True)
        with open("../bot/settings.json", "r") as f:
                data = json.loads(f.read())
                trade_settings = {k: TradeSettings(v, k) for k, v in data['pairs'].items()}
                trade_risk = data['trade_risk']
        pair_settings = trade_settings[pair]
        # print(pair_settings)

        df_an['PAIR'] = pair
        df_an['SPREAD'] = df_an.ask_c - df_an.bid_c
        # print(df_an)


        df_an = RSI(df_an)
        df_an = BollingerBands(df_an, pair_settings.n_ma, pair_settings.n_std)
        df_an = ATR(df_an, pair_settings.atr_period)
        df_an = RSI(df_an, pair_settings.rsi_period)
        df_an = MACD(df_an)
        df_an = momentum_reversal(df_an)
        df_an = apply_patterns(df_an)
        
        df_an['EMA_200'] = df_an.mid_c.ewm(span=200, min_periods=200).mean()
        df_an['GAIN'] = abs(df_an.mid_c - df_an.BB_MA)
       
     
        df_an['TRADE_RISK'] = trade_risk
        df_an['n_ma'] = pair_settings.n_ma
        df_an['n_std'] = pair_settings.n_std
        df_an['maxspread']= pair_settings.maxspread
        df_an['mingain']= pair_settings.mingain
        df_an['riskreward']= pair_settings.riskreward
        df_an['atr_period'] = pair_settings.atr_period
        df_an['atr_multiplier']= pair_settings.atr_multiplier
        df_an['atr_threshold']= pair_settings.atr_threshold
        df_an['rsi_period']= pair_settings.rsi_period
        df_an['rsi_overbought']= pair_settings.rsi_overbought
        df_an['rsi_oversold']= pair_settings.rsi_oversold
        df_an['SIGNAL'] = df_an.apply(apply_signal, axis=1)
        df_an['SL'] = df_an.apply(apply_SL, axis=1)
        df_an['TP'] = df_an.apply(apply_TP, axis=1)
        df_an['LOSS'] = abs(df_an.mid_c - df_an.SL)
        # print(df_an.head(20))

        our_cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c',
        'bid_o', 'bid_h', 'bid_l', 'bid_c', 
        'ask_o', 'ask_h', 'ask_l', 'ask_c',
        'ENGULFING', 'direction', 'EMA_200','MACD',
        'SIGNAL_MD','HIST','RSI_14',f'ATR_{pair_settings.atr_period}', 'BB_UP', 'BB_LW', 
        'BB_MA','BB_Signal','GAIN','SIGNAL','SL','TP','LOSS','SPREAD','PAIR',
        'TRADE_RISK','n_ma','n_std','maxspread','mingain','riskreward',
        'atr_period','atr_multiplier','atr_threshold','rsi_period','rsi_overbought',
        'rsi_oversold','Momentum_Signal']


        df_slim = df_an[our_cols].copy()
        df_slim.dropna(inplace=True)

        df_slim.reset_index(drop=True, inplace=True)
        # print(df_slim.head())

        gt = GuruTester(
        df_slim,
        apply_signal,  
        df_m5,
        use_spread=True
        )

        gt.run_test()
        return gt.df_results.result.sum()


    

In [ ]:
run_pair("EUR_USD")

In [8]:
# res = []
# for p in [ "EUR_USD", "GBP_JPY" ]:
#     res.append(dict(pair=p,res=run_pair(p)))

In [9]:
# for r in res:
#     print(r['pair'], r['res'].result.sum())